In [2]:
import pymongo

RETO 1 – CARGA Y EXPLORACIÓN 
1. Revisa los ficheros adjuntos de la actividad y la documentación 
correspondiente de cada fichero. 
2. Realiza una exploración de los ficheros utilizando MongoDB como motor de 
base de datos. Para ello: 
a. Carga los datos en la BD (utiliza los ficheros CSV o JSON) 
b. Revisa los campos y comprueba la calidad de los datos en cada uno de 
los ficheros. 
c. Encuentra un campo que permita relacionar los ficheros entre sí. 

Apartado A

Insertar los ficheros en cada uno de las colecciones

In [8]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["tarea_nosql"]
mycol_economica = mydb["actividad_economica"]
mycol_licencias = mydb["licencias"]
mycol_locales = mydb["locales"]
mycol_terrazas = mydb["terrazas"]

TypeError: 'Collection' object is not callable. If you meant to call the 'stats' method on a 'Collection' object it is failing because no such method exists.

Apartado B

compruebo la calidad de la coleccion actividad economica

In [13]:
x = mycol_economica.find_one({"id_local" : 20})
e = mycol_licencias.find_one({"id_local" : 7})
y = mycol_locales.find_one({"id_local" : 2})
o = mycol_terrazas.find_one({"id_local" : 270403150})

{'_id': ObjectId('65be21ba17e71cd85abd3344'), 'id_local': 270574777, 'id_distrito_local': 20, 'desc_distrito_local': 'SAN BLAS-CANILLEJAS ', 'id_barrio_local': 2004, 'desc_barrio_local': 'ARCOS               ', 'cod_barrio_local': 4, 'id_seccion_censal_local': 20044, 'desc_seccion_censal_local': 44, 'coordenada_x_local': 0, 'coordenada_y_local': 0, 'id_tipo_acceso_local': 12, 'desc_tipo_acceso_local': 'PC Asociado', 'id_situacion_local': 1, 'desc_situacion_local': 'Abierto', 'id_vial_edificio': 67900, 'clase_vial_edificio': 'CALLE                   ', 'desc_vial_edificio': 'ARCOS DE JALON                                    ', 'id_ndp_edificio': 31026116, 'id_clase_ndp_edificio': 1, 'nom_edificio': 'NUM', 'num_edificio': 34, 'cal_edificio': '  ', 'secuencial_local_PC': 10, 'id_vial_acceso': 67900, 'clase_vial_acceso': 'CALLE                   ', 'desc_vial_acceso': 'ARCOS DE JALON                                    ', 'id_ndp_acceso': 31026116, 'id_clase_ndp_acceso': 1, 'nom_acceso': 'N

Validar los datos de las colecciones

In [ ]:
mydb.command("validate", "actividad_economica")
mydb.command("validate", "licencias")
mydb.command("validate", "locales")
mydb.command("validate", "terrazas")

Apartado C
El campo que permite relcionar entre colecciones es el id_local

RETO 2 – MODELADO DE DATOS 
Después de explorar todos los ficheros y tener clara su estructura, resuelve los 
siguientes tareas. 
Modelo de datos (versión 1) 
1. Propón un modelo de datos basado en documentos que permita almacenar la 
información de los ficheros y consultarla posteriormente. Este modelo 
propuesto puede seguir un patrón de datos visto en este curso (embebidos, 
referenciados o jerárquicos). 
a. Crea una base de datos en MongoDB y almacena los ficheros siguiendo 
tu modelo de datos. 
2. Para mostrar la eficiencia de tu modelo de datos, construye y ejecuta las 
siguientes consultas. 
a. El total de Locales y Terrazas por Distrito y Barrio. 
b. Tipos de licencias y cantidad de licencias por cada tipo.  
c. Listado de locales y terrazas con licencias “En trámite”. 
d. Consulta por sección, división y epígrafe de la actividad comercial de 
locales y terrazas. 
e. Consultar la actividad económica más frecuente por barrio y distrito. 
f. Actualiza los horarios de apertura y cierra de “ciertos” locales 
siguiendo un criterio que tú elijas (explica bien dicho criterio en el 
informe final). 

Modelo de datos (Versión 1)


Crear el indice

In [5]:
pipeline = [
  {"$lookup": {
    "from": "actividad_economica",
    "localField": "id_local",
    "foreignField": "id_local",
    "as": "actividad_economica"
  }},
  {"$lookup": {
    "from": "licencias",
    "localField": "id_local",
    "foreignField": "id_local",
    "as": "licencias"
  }},
  {"$lookup": {
    "from": "terrazas",
    "localField": "id_local",
    "foreignField": "id_local",
    "as": "terrazas"
  }},
  {"$out": "locales_embebidos"}
]
mycol_locales.aggregate(pipeline)

a. El total de Locales y Terrazas por Distrito y Barrio.

In [ ]:
mycol_locales_embebido = mydb["locales_embebidos"]

In [ ]:
resultado = mycol_locales_embebido.aggregate([
    {"$group": {
        "_id": {
            "distrito": "$desc_distrito_local", "barrio": "$desc_barrio_local"
            },
        "total_locales": {
            "$sum": 1
            },
        "total_terrazas": 
        {"$sum": 
         {"$size": "$terrazas"}
        }
    }}
])


for grupo in resultado:
    print(grupo)

b. Tipos de licencias y cantidad de licencias por cada tipo.

In [19]:
resultado = mycol_locales_embebido.aggregate([{"$unwind": "$licencias"},{"$group":
                                               {"_id": "$licencias.desc_tipo_licencia", "total": {"$sum" : 1}
                                                }}])
for grupo in resultado:
    print(grupo)

['ARGANZUELA          ',
 'BARAJAS             ',
 'CARABANCHEL         ',
 'CENTRO              ',
 'CHAMARTIN           ',
 'CHAMBERI            ',
 'CIUDAD LINEAL       ',
 'FUENCARRAL-EL PARDO ',
 'HORTALEZA           ',
 'LATINA              ',
 'MONCLOA-ARAVACA     ',
 'MORATALAZ           ',
 'PUENTE DE VALLECAS  ',
 'RETIRO              ',
 'SALAMANCA           ',
 'SAN BLAS-CANILLEJAS ',
 'TETUAN              ',
 'USERA               ',
 'VICALVARO           ',
 'VILLA DE VALLECAS   ',
 'VILLAVERDE          ']

c. Listado de locales y terrazas con licencias “En trámite”. 

In [ ]:
resultado = mycol_locales_embebido.find({"licencias": {"$elemMatch": {"desc_tipo_situacion_licencia": "En tramitación"}}})

for grupo in resultado:
    print(grupo)

d. Consulta por sección, división y epígrafe de la actividad comercial de 
locales y terrazas. 

In [ ]:
resultado = mycol_locales_embebido.aggregate([
    {"$unwind": "$actividad_economica"},
    {"$group": {
        "_id": {
            "seccion": "$actividad_economica.desc_seccion",
            "division": "$actividad_economica.desc_division",
            "epigrafe": "$actividad_economica.desc_epigrafe"
        },
        "total_locales": {"$sum": 1},
        "total_terrazas": {"$sum": {"$size": "$terrazas"}}
    }}
])

for grupo in resultado:
    print(grupo)

e. Consultar la actividad económica más frecuente por barrio y distrito. 

In [ ]:
resultado = mycol_locales_embebido.aggregate([
    {"$unwind": "$actividad_economica"},
    {"$group": {
        "_id": {
            "distrito": "$desc_distrito_local",
            "barrio": "$desc_barrio_local",
            "actividad": "$actividad_economica.desc_epigrafe"
        },
        "count": {"$sum": 1}
    }},
    {"$sort": {"count": -1}},
    {"$group": {
        "_id": {
            "distrito": "$_id.distrito",
            "barrio": "$_id.barrio"
        },
        "actividad_mas_frecuente": {"$first": "$_id.actividad"},
        "count": {"$first": "$count"}
    }}
])

for grupo in resultado:
    print(grupo)

f. Actualiza los horarios de apertura y cierra de “ciertos” locales 
siguiendo un criterio que tú elijas (explica bien dicho criterio en el 
informe final). 

In [ ]:
mycol_locales_embebido.update_many(
    {"hora_apertura1": {"$lt": "08:00"}},
    {"$set": {"hora_apertura1": "08:00"}}
)

Modelo de datos (versión 2) 

3. Ahora que ya cuentas con un modelo de datos sobre la actividad económica de 
Locales y Terrazas en la ciudad de Madrid, vamos a necesitar conocer la 
actividad relacionada con los alojamientos de los distintos barrios de esta 
misma ciudad. Para incluir esta información, debes realizar lo siguiente: 
a. Revisa la siguiente URL y localiza información sobre los alojamientos 
de Airbnb http://insideairbnb.com/new-york-city/ de la ciudad de 
Madrid. 
b. Debes encontrar un dataset y cargar dicha información en la base de 
datos donde alojas tu modelo de datos. 
c. Asocia la información de alojamientos con la actividad comercial, 
propón algún tipo de mejora de tu modelo de datos para contemplar la 
nueva información.